# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#import "clean cities csv"
file = "../output_data/clean_cities_data.csv"
weather_df = pd.read_csv(file)
weather_df

,City Name,Country Name,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed,Date
0,Mar del Plata,AR,-38.00,-57.56,53.91,100,90,11.41,2020-07-23 22:27:46
1,Aplahoué,BJ,6.93,1.68,72.57,90,100,4.14,2020-07-23 22:31:48
2,Makkah al Mukarramah,SA,21.43,39.83,83.23,58,77,1.88,2020-07-23 22:30:45
3,San Cristobal,VE,7.77,-72.22,82.40,54,75,16.11,2020-07-23 22:31:49
4,Pokaran,IN,26.92,71.92,88.43,61,6,17.22,2020-07-23 22:31:49
...,...,...,...,...,...,...,...,...,...
572,Mvuma,ZW,-19.28,30.53,45.30,89,8,10.56,2020-07-23 22:32:49
573,Guangming,CN,45.33,122.78,84.40,36,2,2.46,2020-07-23 22:32:49
574,Sokoto,NG,13.06,5.24,79.03,73,78,13.35,2020-07-23 22:32:49
575,Sibolga,ID,1.74,98.78,75.72,75,100,3.06,2020-07-23 22:32:49


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#access maps with unique API key
gmaps.configure(api_key = g_key)

In [4]:
#create coordinates
records = weather_df[["Latitude", "Longitude"]].to_records(index=False)
coordinates = list(records)
weights = weather_df['Humidity']
#customize map
figure_layout = {'width': '500px', 'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(coordinates, weights = weights)

#assign marker layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#drop null values
vacation_df = weather_df.dropna(axis=0)
vacation_df

,City Name,Country Name,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed,Date
0,Mar del Plata,AR,-38.00,-57.56,53.91,100,90,11.41,2020-07-23 22:27:46
1,Aplahoué,BJ,6.93,1.68,72.57,90,100,4.14,2020-07-23 22:31:48
2,Makkah al Mukarramah,SA,21.43,39.83,83.23,58,77,1.88,2020-07-23 22:30:45
3,San Cristobal,VE,7.77,-72.22,82.40,54,75,16.11,2020-07-23 22:31:49
4,Pokaran,IN,26.92,71.92,88.43,61,6,17.22,2020-07-23 22:31:49
...,...,...,...,...,...,...,...,...,...
572,Mvuma,ZW,-19.28,30.53,45.30,89,8,10.56,2020-07-23 22:32:49
573,Guangming,CN,45.33,122.78,84.40,36,2,2.46,2020-07-23 22:32:49
574,Sokoto,NG,13.06,5.24,79.03,73,78,13.35,2020-07-23 22:32:49
575,Sibolga,ID,1.74,98.78,75.72,75,100,3.06,2020-07-23 22:32:49


In [6]:
#select only ideal cities
#Criteria - Temperature >70 <85 degrees, cloudiness <50, humidity <60

ideal_cities_df = vacation_df.loc[(vacation_df["Temperature"] >=75) & 
                                  (vacation_df["Temperature"] <=80) & 
                                  (vacation_df["Cloudiness"] <50) & 
                                  (vacation_df["Humidity"] <50)]
ideal_cities_df

,City Name,Country Name,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed,Date
164,Chapais,CA,49.78,-74.85,77.00,36,1,10.29,2020-07-23 22:32:06
201,Porto Murtinho,BR,-21.70,-57.88,77.65,34,0,4.47,2020-07-23 22:32:10
330,South Lake Tahoe,US,38.93,-119.98,76.06,33,1,8.08,2020-07-23 22:32:06
340,Nioaque,BR,-21.14,-55.83,75.06,35,0,3.22,2020-07-23 22:32:25
341,Pontes e Lacerda,BR,-15.23,-59.34,78.24,32,0,2.62,2020-07-23 22:32:25
388,Athens,GR,37.98,23.72,79.77,36,0,9.17,2020-07-23 22:29:31
464,Asilah,MA,35.47,-6.03,78.80,47,0,2.24,2020-07-23 22:32:37


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#create hotel df
hotel_df = ideal_cities_df.filter(["City Name", "Country Name", "Latitude", "Longitude"], axis=1)
#rename column headers to match requirements below for map
hotel_df.rename(columns={'City Name':'City',
                         'Country Name': 'Country',
                          'Latitude':'Lat',
                          'Longitude':'Lng'},inplace=True)
hotel_df["Hotel Name"] = ""
hotel_df


,City,Country,Lat,Lng,Hotel Name
164,Chapais,CA,49.78,-74.85,
201,Porto Murtinho,BR,-21.70,-57.88,
330,South Lake Tahoe,US,38.93,-119.98,
340,Nioaque,BR,-21.14,-55.83,
341,Pontes e Lacerda,BR,-15.23,-59.34,
388,Athens,GR,37.98,23.72,
464,Asilah,MA,35.47,-6.03,


In [8]:
#query from Google Places API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#loop through coordinates to get responses
for index,row in hotel_df.iterrows():

    
    #get location from df
    latitude = row["Lat"]
    longitude = row["Lng"]
    city = row["City"]
    
    #add to params
    locations = f"{latitude},{longitude}"
    
    #set up params
    params = {"location":locations,
               "keyword": "lodging",
                "radius":5000, 
                #"rankby":"distance",
                   "key": g_key,}
    
    #create URL for request
    response = requests.get(base_url, params=params).json()
    
    #extract results - print for testing
    #print(json.dumps(response, indent=4, sort_keys=True))
    
    #in case there are errors
    try:
        print("------------------------------------")
        print(f' Hotel found within 5000m of {city}')
        print("------------------------------------")
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("------------------------------------")
        print("Not result found... skipping.")
        print("------------------------------------")
        hotel_df.loc[index, "Hotel Name"] = "N/A"
        

    

------------------------------------
 Hotel found within 5000m of Chapais
------------------------------------
------------------------------------
 Hotel found within 5000m of Porto Murtinho
------------------------------------
------------------------------------
 Hotel found within 5000m of South Lake Tahoe
------------------------------------
------------------------------------
 Hotel found within 5000m of Nioaque
------------------------------------
------------------------------------
 Hotel found within 5000m of Pontes e Lacerda
------------------------------------
------------------------------------
 Hotel found within 5000m of Athens
------------------------------------
------------------------------------
 Hotel found within 5000m of Asilah
------------------------------------


In [9]:
#view hotel names on df
hotel_df

,City,Country,Lat,Lng,Hotel Name
164,Chapais,CA,49.78,-74.85,Hôtel Opémiska
201,Porto Murtinho,BR,-21.70,-57.88,Pousada do Pescador
330,South Lake Tahoe,US,38.93,-119.98,The Lodge at Edgewood Tahoe
340,Nioaque,BR,-21.14,-55.83,Nioaque Novo Hotel
341,Pontes e Lacerda,BR,-15.23,-59.34,Hotel França
388,Athens,GR,37.98,23.72,Herodion Hotel
464,Asilah,MA,35.47,-6.03,Asilah 32


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(
    locations,
    info_box_content=hotel_info,
    display_info_box=True)

# Add layer
fig.add_layer(marker_layer)

#update figure display
#gmaps.figure( zoom_level=16, center=(36.0999,80.2442)) #coordinates are for Winston Salem, NC. I wanted to default the zoom if I could.

# Display the map
fig


Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))